In [1]:
!pip3 install msprime

     |████████████████████████████████| 1.4MB 3.8MB/s eta 0:00:01
     |████████████████████████████████| 40kB 22.3MB/s eta 0:00:01
     |████████████████████████████████| 460kB 37.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 20.2MB/s eta 0:00:01
     |████████████████████████████████| 122kB 18.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 12.1MB/s eta 0:00:01
     |████████████████████████████████| 153kB 45.9MB/s eta 0:00:01
  Found existing installation: attrs 17.4.0
    Uninstalling attrs-17.4.0:
      Successfully uninstalled attrs-17.4.0


In [8]:
import msprime
import numpy as np

In [20]:

# Define parameters
population_size = 100000
outgroup_size = 1   # Single individual for the outgroup
num_individuals = 10  # Number of diploid individuals
chromosome_length = 10_000_000
mutation_rate = 1e-8
recombination_rate = 1e-8
divergence_time = 10000000  # Time of divergence in generations for the outgroup

# Setup demography, including an outgroup
demography = msprime.Demography()
demography.add_population(name="ingroup", initial_size=population_size)
demography.add_population(name="outgroup", initial_size=outgroup_size)
demography.add_population_split(time=divergence_time, derived=["ingroup"], ancestral="outgroup")

# Simulate the tree sequence
tree_sequence = msprime.sim_ancestry(
    samples={"ingroup": num_individuals, "outgroup": 1},  # One haploid outgroup
    ploidy=2,  # Haploid to manage samples as individual haplotypes
    demography=demography,
    sequence_length=chromosome_length,
    recombination_rate=recombination_rate,
    random_seed=42
)

# Introduce mutations on the tree sequence
mutated_ts = msprime.sim_mutations(
    tree_sequence,
    rate=mutation_rate,
    random_seed=42
)

# Extract mutational data and polarize
ancestor_map = {}
for variant in mutated_ts.variants():
    # Using outgroup sample index, find its allele
    outgroup_allele = variant.genotypes[-1]
    ancestral_allele = variant.alleles[outgroup_allele]
    ancestor_map[variant.site.position] = ancestral_allele

# Write the VCF
output_vcf = "simulated_data.vcf"
with open(output_vcf, "w") as vcf_file:
    mutated_ts.write_vcf(vcf_file)

# Reading and updating VCF to add ancestral info
with open(output_vcf, "r") as vcf_input, open("polarized_" + output_vcf, "w") as vcf_output:
    for line in vcf_input:
        if line.startswith("#"):
            # Add an extra header line for ancestral state information
            if line.startswith("#CHROM"):
                vcf_output.write("##INFO=<ID=AA,Number=1,Type=String,Description=\"Ancestral Allele\">\n")
            vcf_output.write(line)
        else:
            # Find the position and look up the ancestral allele for that position
            fields = line.strip().split("\t")
            pos = int(fields[1])
            ancestral_allele = ancestor_map.get(pos, '.')

            # Append the ancestral allele info to the INFO field
            fields[7] = f"{fields[7]};AA={ancestral_allele}"
            vcf_output.write("\t".join(fields) + "\n")

print("Polarization complete. Polarized VCF file 'polarized_simulated_data.vcf' generated.")

Polarization complete. Polarized VCF file 'polarized_simulated_data.vcf' generated.


In [11]:
import msprime

# Define the simulation parameters
population_size = 1000  # Effective population size
chromosome_length = 10_000_000  # 10 Mb chromosome
mutation_rate = 1e-8  # Mutation rate per base per generation
recombination_rate = 1e-8  # Recombination rate per base per generation
num_individuals = 20  # Number of diploid individuals

# Simulate the ancestral recombination graph
tree_sequence = msprime.simulate(
    sample_size=num_individuals * 2,  # Diploid individuals, so * 2 for haploids
    length=chromosome_length,  # Length of the simulated chromosome
    Ne=population_size,  # Effective population size
    mutation_rate=mutation_rate,
    recombination_rate=recombination_rate
)

# Write the output to a VCF file
with open("simulated_data.vcf", "w") as vcf_file:
    tree_sequence.write_vcf(vcf_file, 2)  # 2 is the number of alleles per site (diploid)

# Note: This will use msprime's internal methods to write out the VCF
print("Simulation complete. VCF file 'simulated_data.vcf' generated.")

Simulation complete. VCF file 'simulated_data.vcf' generated.
